In [1]:
# install octave
!sudo apt-get -qq update
!sudo apt-get -qq install octave octave-signal liboctave-dev

# install oct2py that compatible with colab
import os

from pkg_resources import get_distribution

os.system(
    f"pip install -qq"
    f" ipykernel=={get_distribution('ipykernel').version}"
    f" ipython=={get_distribution('ipython').version}"
    f" tornado=={get_distribution('tornado').version}"
    f" oct2py"
)

# install packages
!pip install -qq matpower matpowercaseframes

In [2]:
import oct2py

import matpower

print(f"oct2py version: {oct2py.__version__}")
print(f"matpower version: {matpower.__version__}")

oct2py version: 5.7.2
matpower version: 8.0.0.2.2.2


In [3]:
import os

from matpowercaseframes import CaseFrames

from matpower import path_matpower, start_instance

## Inspect data

In [4]:
path = os.path.join(path_matpower, "data/case9.m")

In [5]:
m = start_instance()

In [6]:
cf = CaseFrames(path, load_case_engine=m)
cf.infer_numpy()  # for better print, convert floats to ints
cf.gen

,GEN_BUS,PG,QG,QMAX,QMIN,VG,MBASE,GEN_STATUS,PMAX,PMIN,...,PC2,QC1MIN,QC1MAX,QC2MIN,QC2MAX,RAMP_AGC,RAMP_10,RAMP_30,RAMP_Q,APF
1,1,72.3,27.03,300,-300,1.040,100,1,250,10,...,0,0,0,0,0,0,0,0,0,0
2,2,163.0,6.54,300,-300,1.025,100,1,300,10,...,0,0,0,0,0,0,0,0,0,0
3,3,85.0,-10.95,300,-300,1.025,100,1,270,10,...,0,0,0,0,0,0,0,0,0,0


## Run base

In [7]:
mpc = m.runpf(cf.to_mpc(), verbose=True)
cf_sol = CaseFrames(mpc)


MATPOWER Version 8.0, 17-May-2024
Power Flow -- AC-polar-power formulation

Newton's method converged in 4 iterations.
PF successful

Converged in 0.13 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              9     Total Gen Capacity     820.0        -900.0 to 900.0
Generators         3     On-line Capacity       820.0        -900.0 to 900.0
Committed Gens     3     Generation (actual)    319.6              22.8
Loads              3     Load                   315.0             115.0
  Fixed            3       Fixed                315.0             115.0
  Dispatchable     0       Dispatchable          -0.0 of -0.0      -0.0
Shunts             0     Shunt (inj)             -0.0               0.0
Branches           9     Losses (I^2 * Z)         4.64             48.38
Transformers    

## Copy and editing gen

In [8]:
new_gen = cf_sol.gen.loc[1, :].copy()  # copy cf_sol.gen.index 1
new_gencost = cf_sol.gencost.loc[1, :].copy()  # copy cf_sol.gencost.index 1

new_gen.rename(4, inplace=True)
new_gencost.rename(4, inplace=True)

new_gen = cf_sol.gen.loc[1, :].copy()  # copy cf_sol.gen.index 1
new_gen["GEN_BUS"] = 8  # connect place to bus 8

cf_sol.gen = cf_sol.gen._append([new_gen])
cf_sol.gencost = cf_sol.gencost._append([new_gencost])
cf_sol.gen

,GEN_BUS,PG,QG,QMAX,QMIN,VG,MBASE,GEN_STATUS,PMAX,PMIN,...,PC2,QC1MIN,QC1MAX,QC2MIN,QC2MAX,RAMP_AGC,RAMP_10,RAMP_30,RAMP_Q,APF
1,1.0,71.641021,27.045924,300.0,-300.0,1.040,100.0,1.0,250.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,163.000000,6.653660,300.0,-300.0,1.025,100.0,1.0,300.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,85.000000,-10.859709,300.0,-300.0,1.025,100.0,1.0,270.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8.0,71.641021,27.045924,300.0,-300.0,1.040,100.0,1.0,250.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
mpc = m.runpf(cf.to_mpc(), verbose=True)
cf_sol = CaseFrames(mpc)


MATPOWER Version 8.0, 17-May-2024
Power Flow -- AC-polar-power formulation

Newton's method converged in 4 iterations.
PF successful

Converged in 0.08 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              9     Total Gen Capacity     820.0        -900.0 to 900.0
Generators         3     On-line Capacity       820.0        -900.0 to 900.0
Committed Gens     3     Generation (actual)    319.6              22.8
Loads              3     Load                   315.0             115.0
  Fixed            3       Fixed                315.0             115.0
  Dispatchable     0       Dispatchable          -0.0 of -0.0      -0.0
Shunts             0     Shunt (inj)             -0.0               0.0
Branches           9     Losses (I^2 * Z)         4.64             48.38
Transformers    

In [10]:
cf_sol.gen

,GEN_BUS,PG,QG,QMAX,QMIN,VG,MBASE,GEN_STATUS,PMAX,PMIN,...,PC2,QC1MIN,QC1MAX,QC2MIN,QC2MAX,RAMP_AGC,RAMP_10,RAMP_30,RAMP_Q,APF
1,1.0,71.641021,27.045924,300.0,-300.0,1.040,100.0,1.0,250.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,163.000000,6.653660,300.0,-300.0,1.025,100.0,1.0,300.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,85.000000,-10.859709,300.0,-300.0,1.025,100.0,1.0,270.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
